## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

---
## Pipeline for video analysis of lane lines

In [2]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
from moviepy.editor import VideoFileClip
%matplotlib
avg = 0
avg_offset=0
cashed_curv=[]
cashed_offset=[]

def process_image(image):
    
    global avg
    global avg_offset
    global cashed_curv
    global cashed_offset
    global mtx
    global dist
    
    # Rectification of perspective
    #image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    image =  cv2.undistort(image, mtx, dist, None,mtx)
    image = cv2.copyMakeBorder(image,0,0,320,320,cv2.BORDER_CONSTANT,value=(0,0,0))
    src = np.float32([[0,720],[863,450],[1057,450],[1920,720]])
    dst = np.float32([[0,720],[0,0],[1920,0],[1920,720]])
    
    M = cv2.getPerspectiveTransform(src, dst)
    Minv = cv2.getPerspectiveTransform(dst, src)

    warped1 = cv2.warpPerspective(image, M, image[:,:,0].shape[::-1], flags=cv2.INTER_LINEAR)
    warped2 = cv2.warpPerspective(image, M, image[:,:,1].shape[::-1], flags=cv2.INTER_LINEAR)
    warped3 = cv2.warpPerspective(image, M, image[:,:,2].shape[::-1], flags=cv2.INTER_LINEAR)

    r_channel = warped3[:,:,0]
    lab = cv2.cvtColor(warped3,cv2.COLOR_RGB2LAB)
    hls = cv2.cvtColor(warped3,cv2.COLOR_RGB2HLS)

    s = hls[:,:,2]
    l = lab[:,:,0]

    b = lab[:,:,2]

    thresh_b = (145, 255)
    binary_b = np.zeros_like(b)
    binary_b[(b > thresh_b[0]) & (b <= thresh_b[1])] = 1

    thresh_r = (220, 255)
    binary_r = np.zeros_like(s)
    binary_r[(r_channel > thresh_r[0]) & (r_channel <= thresh_r[1])] = 1

    thresh_s = (160, 255)
    binary_s = np.zeros_like(s)
    binary_s[(s > thresh_s[0]) & (s <= thresh_s[1])] = 1

    thresh_l = (215, 255)
    binary_l = np.zeros_like(l)
    binary_l[(l > thresh_l[0]) & (l <= thresh_l[1])] = 1

    combined = cv2.max(binary_l,binary_b)

    ##################################################
    ######## fitting polynomial of lane ##############

    # Assuming you have created a warped binary image called "binary_warped"
    # Take a histogram of the bottom half of the image
    binary_warped = combined
    histogram = np.sum(binary_warped[  int(binary_warped.shape[0]/2) :,:], axis=0)
    # Create an output image to draw on and  visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]/2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # Choose the number of sliding windows
    nwindows = 9
    # Set height of windows
    window_height = np.int(binary_warped.shape[0]/nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated for each window
    leftx_current = leftx_base
    rightx_current = rightx_base
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 30
    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high),
        (0,255,0), 2) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),(win_xright_high,win_y_high),
        (0,255,0), 2) 
        # Identify the nonzero pixels in x and y within the window
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds] 
    
    # Fit a second order polynomial to each
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)

    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

    out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]
    #plt.imshow(out_img)
    #plt.plot(left_fitx, ploty, color='yellow')
    #plt.plot(right_fitx, ploty, color='yellow')
    #plt.xlim(0, 1280)
    #plt.ylim(720, 0)
    
    ################ Draw on the raw image #############
    
    # Create an image to draw the lines on
    warp_zero = np.zeros_like(combined).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))
    
    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, (image.shape[1], image.shape[0])) 
    # Combine the result with the original image
    result = cv2.addWeighted(image, 1, newwarp, 0.3, 0)
    cropped = result[:,320:1600].copy()
    ym_per_pix = 34/720 # meters per pixel in y dimension
    xm_per_pix = 3.5/900 # meters per pixel in x dimension
    y_eval = np.max(ploty)
    # Fit new polynomials to x,y in world space
    right_fit_cr = np.polyfit(righty*ym_per_pix, rightx*xm_per_pix, 2)
    left_fit_cr = np.polyfit(lefty*ym_per_pix, leftx*xm_per_pix, 2)
    # Calculate the new radii of curvature
    left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
    
    offset = (((rightx_base + leftx_base)/2)-midpoint)*(3.7/900)*100
    cashed_offset.append(offset)
    
    ## Sanity check
    if(0.5<left_curverad/right_curverad<2 ):
        cashed_curv.append( (left_curverad+right_curverad)/2)
    # Now our radius of curvature is in meters
    #print(left_curverad, 'm', right_curverad, 'm')
    font = cv2.FONT_HERSHEY_SIMPLEX
    
    
    if len(cashed_curv)>=5:
        avg = int(sum(cashed_curv)/len(cashed_curv))
        avg_offset = int(sum(cashed_offset)/len(cashed_offset))
        cv2.putText(cropped,"Radius of curvature = {0} m".format(avg),(50,100),font,1,(255,255,255),2,cv2.LINE_AA)
        if(avg_offset>=0 ):
            cv2.putText(cropped,"Distance from lane center = {0} cm to the left".format(avg_offset),(50,170), font,1,(255,255,255),2,cv2.LINE_AA)
        else :
            avg_offset = -1*avg_offset
            cv2.putText(cropped,"Distance from lane center = {0} cm to the right".format(avg_offset),(50,170), font,1,(255,255,255),2,cv2.LINE_AA)
       
        cashed_curv=[]
        cashed_offset=[]
    else:
        cv2.putText(cropped,"Radius of curvature = {0} m".format(avg),(50,100), font,1,(255,255,255),2,cv2.LINE_AA)
        if(avg_offset>=0 ):
            cv2.putText(cropped,"Distance from lane center = {0} cm to the left".format(avg_offset),(50,170), font,1,(255,255,255),2,cv2.LINE_AA)
        else :
            avg_offset = -1*avg_offset
            cv2.putText(cropped,"Distance from lane center = {0} cm to the right".format(avg_offset),(50,170), font,1,(255,255,255),2,cv2.LINE_AA)
       
    return cropped




# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*9,3), np.float32)
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane.

# Make a list of calibration images
images = glob.glob('../camera_cal/calibration*.jpg')


# Step through the list and search for chessboard corners
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

# Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (9,6),None)

# If found, add object points, image points
    if ret == True:
        objpoints.append(objp)
        imgpoints.append(corners)

# Draw and display the corners
returned, mtx, dist, rvecs, tvecs =cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1],None,None)
    


video_output = 'test_videos_output/out_vid.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip1 = VideoFileClip("test_videos_output/project_video.mp4").subclip(0,5)
clip1 = VideoFileClip("test_videos_output/project_video.mp4")
myclip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time myclip.write_videofile(video_output, audio=False)



Using matplotlib backend: Qt5Agg
[MoviePy] >>>> Building video test_videos_output/outvid4.mp4
[MoviePy] Writing video test_videos_output/outvid4.mp4


100%|█████████████████████████████████████▉| 1260/1261 [05:48<00:00,  3.92it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/outvid4.mp4 

Wall time: 5min 50s
